In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gym
import os
import copy
import math
import random
import statistics
from collections import deque
from collections import namedtuple
import matplotlib.pyplot as plt

In [ ]:
'''ENV_NAME List of MuJoCo Tasks'''
# InvertedDoublePendulum-v2
# Hopper-v2
# HalfCheetah-v2


ENV_NAME = 'HalfCheetah-v2'
SEED = 0

# Set environment & seeds
env = gym.make(ENV_NAME)
env.seed(SEED)
env.action_space.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

# Set state-action dimension
PLANT_STATE_DIM = env.observation_space.shape[0]
CHANNEL_STATE_DIM = 1
AoI_DIM = 1

CONTROL_ACTION_DIM = env.action_space.shape[0]
SCHEDULE_ACTION_DIM = 1 
MAX_ACTION = float(env.action_space.high[0])

# Transmission scheduler action: 1-transmit; 0-not transmit
TRANSMISSION_ACTION_LIST = np.array([0,1])

# Set NNs hyperparameters of estimator, intelligent controller (actor, critic) and transmission scheduler (DQN)
ACTOR_INPUT_DIM = PLANT_STATE_DIM + 2*CHANNEL_STATE_DIM + AoI_DIM # (h,s) → ACT → a
ACTOR_OUTPUT_DIM = CONTROL_ACTION_DIM

CRITIC_INPUT_DIM = (PLANT_STATE_DIM + 2*CHANNEL_STATE_DIM + AoI_DIM) + CONTROL_ACTION_DIM
CRITIC_OUTPUT_DIM = 1

ESTIMATOR_INPUT_DIM = PLANT_STATE_DIM + CONTROL_ACTION_DIM
ESTIMATOR_OUTPUT_DIM = PLANT_STATE_DIM

DQN_INPUT_DIM = PLANT_STATE_DIM + CHANNEL_STATE_DIM + AoI_DIM
DQN_OUTPUT_DIM = 2

HIDDEN_SIZE = 128

# Set history length
LEN = 3

SCHEDULER_OW_DIM = LEN*(DQN_INPUT_DIM+SCHEDULE_ACTION_DIM)
CONTROLLER_OW_DIM = LEN*(ACTOR_INPUT_DIM+CONTROL_ACTION_DIM)

# Set network training hyperparameters
TIMESTEPS_BEFORE_TRAIN = 25e3
PRE_TRAINING_TIMESTEPS = 2e5
MAX_TIMESTEPS = 2e6
GAMMA = 0.99
TAU = 0.005
LR = 3e-4
LR_DYNAMICS = 1e-3
EPSILON = 0.9

EXPL_NOISE_STD = 0.1
TARGET_ACTOR_NOISE_STD = 0.2
TARGET_ACTOR_NOISE_CLIP = 0.5
ACTOR_UPDATE_DELAY = 2

MEMORY_CAPACITY = int(2e6)
BATCH_SIZE = 100

EVAL_EPISODES = 10
EVAL_FREQ = 5e3

# Set WNCS hyperparameters
LS = 0.05 # initial uplink (sensor-controller) channel loss rate
LA = 0.05 # initial downlink (controller-actuator) channel loss rate
SENSOR_NOISE_STD = 0.01 
COMM_COST = 10

# Set experience replay hyperparameters
ALPHA_IS = 1 # rank-based importance sampling
ALPHA_UN = 0 # uniform sampling
BETA = 1

MEMORY_SIZE = int(2e5)
SORT_FREQ = int(2e5)

In [ ]:
'''Initialize NNs'''
# Estimator
# Intelligent controller (actor, critic) 
# Transmission scheduler (DQN) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Estimator(nn.Module):
    def __init__(self):
        super(Estimator, self).__init__()
        
        self.gru = nn.GRU(ESTIMATOR_INPUT_DIM, HIDDEN_SIZE)
        self.l1 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.l2 = nn.Linear(HIDDEN_SIZE, ESTIMATOR_OUTPUT_DIM)
        

    def forward(self, state, action):
        self.gru.flatten_parameters()
        sa = torch.cat([state, action], 1)
        
        s_gru, _ = self.gru(sa.view(len(sa), 1, -1))
        s = s_gru.view(len(sa), -1)
        
        s = F.relu(self.l1(s))
        
        return self.l2(s)
    
class Actor(nn.Module):
    def __init__(self):
        super(Actor, self).__init__()
        
        self.pre = nn.Linear(CONTROLLER_OW_DIM, HIDDEN_SIZE)
        self.gru = nn.GRU(HIDDEN_SIZE, HIDDEN_SIZE)
        
        self.l1 = nn.Linear(ACTOR_INPUT_DIM, HIDDEN_SIZE)
        
        self.l2 = nn.Linear(2*HIDDEN_SIZE, HIDDEN_SIZE)
        self.l3 = nn.Linear(HIDDEN_SIZE, ACTOR_OUTPUT_DIM)

        self.max_action = MAX_ACTION


    def forward(self, ow, state):
        self.gru.flatten_parameters()
        
        a_his = F.relu(self.pre(ow))
        a_gru, _ = self.gru(a_his.view(len(ow), 1, -1))
        a_his = a_gru.view(len(ow), -1)
        
        a_cur = F.relu(self.l1(state))
        
        a = torch.cat([a_his, a_cur], -1)
        a = F.relu(self.l2(a))
        
        return self.max_action * torch.tanh(self.l3(a))


class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()

        # Q1
        self.pre1 = nn.Linear(CONTROLLER_OW_DIM, HIDDEN_SIZE)
        self.gru1 = nn.GRU(HIDDEN_SIZE, HIDDEN_SIZE)
        
        self.l1 = nn.Linear(CRITIC_INPUT_DIM, HIDDEN_SIZE)
        
        self.l2 = nn.Linear(2*HIDDEN_SIZE, HIDDEN_SIZE)
        self.l3 = nn.Linear(HIDDEN_SIZE, CRITIC_OUTPUT_DIM)

        # Q2
        self.pre2 = nn.Linear(CONTROLLER_OW_DIM, HIDDEN_SIZE)
        self.gru2 = nn.GRU(HIDDEN_SIZE, HIDDEN_SIZE)
        
        self.l4 = nn.Linear(CRITIC_INPUT_DIM, HIDDEN_SIZE)
        
        self.l5 = nn.Linear(2*HIDDEN_SIZE, HIDDEN_SIZE)
        self.l6 = nn.Linear(HIDDEN_SIZE, CRITIC_OUTPUT_DIM)
        
        
    def forward(self, ow, state, action):
        self.gru1.flatten_parameters()
        self.gru2.flatten_parameters()
        sa = torch.cat([state, action], 1)
        
        # Q1
        q1_his = F.relu(self.pre1(ow))
        q1_gru, _ = self.gru1(q1_his.view(len(ow), 1, -1))
        q1_his = q1_gru.view(len(ow), -1)
        
        q1_cur = F.relu(self.l1(sa))
        
        q1 = torch.cat([q1_his, q1_cur], -1)
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        
        # Q2
        q2_his = F.relu(self.pre2(ow))
        q2_gru, _ = self.gru2(q2_his.view(len(ow), 1, -1))
        q2_his = q2_gru.view(len(ow), -1)
        
        q2_cur = F.relu(self.l4(sa))
        
        q2 = torch.cat([q2_his, q2_cur], -1)
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        
        return q1, q2


    def Q1(self, ow, state, action):
        self.gru1.flatten_parameters()
        sa = torch.cat([state, action], 1)
        
        q1_his = F.relu(self.pre1(ow))
        q1_gru, _ = self.gru1(q1_his.view(len(ow), 1, -1))
        q1_his = q1_gru.view(len(ow), -1)
        
        q1_cur = F.relu(self.l1(sa))
        
        q1 = torch.cat([q1_his, q1_cur], -1)
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        
        return q1
        

class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        
        self.pre = nn.Linear(SCHEDULER_OW_DIM, HIDDEN_SIZE)
        self.gru = nn.GRU(HIDDEN_SIZE, HIDDEN_SIZE)
        
        self.l1 = nn.Linear(DQN_INPUT_DIM, HIDDEN_SIZE)
        
        self.l2 = nn.Linear(2*HIDDEN_SIZE, HIDDEN_SIZE)
        self.l3 = nn.Linear(HIDDEN_SIZE, DQN_OUTPUT_DIM)


    def forward(self, ow, state):
        self.gru.flatten_parameters()
        
        q_his = F.relu(self.pre(ow))
        q_gru, _ = self.gru(q_his.view(len(ow), 1, -1))
        q_his = q_gru.view(len(ow), -1)
        
        q_cur = F.relu(self.l1(state))
        
        q = torch.cat([q_his, q_cur], -1)
        q = F.relu(self.l2(q))
        
        return self.l3(q)

In [ ]:
'''Tailored TD3 Algorithm'''
# select_action: generate control signal by actor
# select_q: generate Q-value by critic
# predict_state: generate predicted state by estimator
# schedule_transmission & control_transmission: generate transmission scheduler action based on DQN output
# train: network training of estimator, intelligent controller (actor, critic) and transmission scheduler (DQN)


class Tailored_TD3(object):
    def __init__(self):
        self.estimator = Estimator().to(device)
        self.estimator_optimizer = torch.optim.Adam(self.estimator.parameters(), lr=LR_DYNAMICS)
        
        self.dqn = DQN().to(device)
        self.dqn_target = copy.deepcopy(self.dqn)
        self.dqn_optimizer = torch.optim.Adam(self.dqn.parameters(), lr=LR)

        self.actor = Actor().to(device)
        self.actor_target = copy.deepcopy(self.actor)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=LR)

        self.critic = Critic().to(device)
        self.critic_target = copy.deepcopy(self.critic)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=LR)

        self.max_action = MAX_ACTION
        self.discount = GAMMA
        self.tau = TAU
        self.policy_noise = TARGET_ACTOR_NOISE_STD
        self.noise_clip = TARGET_ACTOR_NOISE_CLIP
        self.policy_freq = ACTOR_UPDATE_DELAY

        self.total_it = 0


    def select_action(self, ow, state):
        ow = torch.FloatTensor(ow.reshape(1, -1)).to(device)
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(ow, state).cpu().data.numpy().flatten()
    
    
    def select_q(self, ow, state, action):
        ow = torch.FloatTensor(ow.reshape(1, -1)).to(device)
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        action = torch.FloatTensor(action.reshape(1, -1)).to(device)
        return self.critic.Q1(ow, state, action).cpu().data.numpy().flatten()

    
    def predict_state(self, state, action):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        action = torch.FloatTensor(action.reshape(1, -1)).to(device)
        return self.estimator(state, action).cpu().data.numpy().flatten()
    
    
    def schedule_transmission(self, ow, state):
        ow = torch.FloatTensor(ow.reshape(1, -1)).to(device)
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        
        schedule_action_value = self.dqn(ow, state)
        schedule_action = torch.max(schedule_action_value, 1)[1].cpu().data.numpy()[0]
        return np.array([schedule_action])
    
    
    def control_transmission(self, schedule_action):
        transmission_action_idx = int(schedule_action)
        transmission_action = TRANSMISSION_ACTION_LIST[transmission_action_idx]
        return transmission_action

    
    def train(self, estimator_replay_buffer, scheduler_replay_buffer, controller_replay_buffer):
        # Sort memory every n steps
        if(self.total_it % SORT_FREQ == 0):
            controller_replay_buffer.sort_priorities()
            
        self.total_it += 1
        
        # Sample replay buffer (Estimator update)
        state_E, action_E, reward_E, next_state_E, not_done_E, idxs_E, sampling_weights_E = estimator_replay_buffer.sample()
        sampling_weights_E_sqrt = sampling_weights_E.sqrt()

        # Compute estimator loss
        estimator_loss = F.mse_loss(self.estimator(state_E, action_E), next_state_E)

        # Optimize the estimator
        self.estimator_optimizer.zero_grad()
        estimator_loss.backward()
        self.estimator_optimizer.step()
        
        # Sample replay buffer (Actor-critic update)
        ow, state, action, reward, next_ow, next_state, not_done, idxs, sampling_weights = controller_replay_buffer.sample_ow()
        sampling_weights_sqrt = sampling_weights.sqrt()

        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (torch.randn_like(action) * self.policy_noise).clamp(-self.noise_clip, self.noise_clip)
            next_action = (self.actor_target(next_ow, next_state) + noise).clamp(-self.max_action, self.max_action)

            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target(next_ow, next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + not_done * self.discount * target_Q

        # Get current Q estimates
        current_Q1, current_Q2 = self.critic(ow, state, action)

        # Compute critic loss
        critic_loss = F.mse_loss(sampling_weights_sqrt*current_Q1, 
                                 sampling_weights_sqrt*target_Q) + F.mse_loss(sampling_weights_sqrt*current_Q2, 
                                                                              sampling_weights_sqrt*target_Q)
        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:

            # Compute actor loss
            actor_loss = -self.critic.Q1(ow, state, self.actor(ow, state)).mean()

            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Update target networks
            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        # Update sampling priority according to AoI-based experience replay
        update_idxs = idxs.cpu().numpy().astype(int)
        TD_error = ((current_Q1 - target_Q)**2 + (current_Q2 - target_Q)**2).detach().cpu().numpy().flatten().astype(float)
        controller_replay_buffer.update_priorities(update_idxs, TD_error)
        
        if(self.total_it >= PRE_TRAINING_TIMESTEPS - TIMESTEPS_BEFORE_TRAIN):
            # Sample replay buffer (DQN update)
            ow_S, state_S, action_S, reward_S, next_ow_S, next_state_S, not_done_S, idxs_S, sampling_weights_S = scheduler_replay_buffer.sample_ow()

            with torch.no_grad():
                # Compute the target Q value
                target_Q_S = self.dqn_target(next_ow_S, next_state_S)
                target_Q_S = reward_S + not_done_S * self.discount * target_Q_S.max(1)[0].unsqueeze(1)

            # Get current Q estimates
            action_S = action_S.long()
            current_Q_S = self.dqn(ow_S, state_S).gather(1, action_S)

            # Compute dqn loss
            dqn_loss = F.mse_loss(current_Q_S, target_Q_S)

            # Optimize the dqn
            self.dqn_optimizer.zero_grad()
            dqn_loss.backward()
            self.dqn_optimizer.step()
            
            if self.total_it % 100 == 0:
                # Update target networks
                for param, target_param in zip(self.dqn.parameters(), self.dqn_target.parameters()):
                    target_param.data.copy_(param.data)
                    
            # Update sampling priority according to AoI-based experience replay
            update_idxs_S = idxs_S.cpu().numpy().astype(int)
            TD_error_S = ((current_Q_S - target_Q_S)**2).detach().cpu().numpy().flatten().astype(float)
            scheduler_replay_buffer.update_priorities(update_idxs_S, TD_error_S)

In [ ]:
'''Initialize Experience Replay Buffer'''
# PrioritizedExperienceReplayBuffer: transitions in the format of [state, action, reward, next_state, done]
# PrioritizedExperienceReplayBuffer_OW: transitions in the format of [history, state, action, reward, next_history, next_state, done]

Experience = namedtuple("Experience", ("states", "actions", "rewards", "next_states", "dones"))
Experience_OW = namedtuple("Experience_OW", ("ows", "states", "actions", "rewards", "next_ows", "next_states", "dones")) 

class PrioritizedExperienceReplayBuffer:

    def __init__(self, alpha=ALPHA_UN, beta=BETA):
      
        self._batch_size = BATCH_SIZE
        self._buffer_size = MEMORY_CAPACITY
        self._buffer_length = 0 # current buffer length
        self._buffer = np.empty(self._buffer_size, dtype=[("priority", np.float32), ("experience", Experience)])
        
        self._alpha = alpha
        self._beta = beta
        self._random_state = np.random.RandomState()
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def buffer_len(self):
        return self._buffer_length
    
    def is_full(self) -> bool:
        return self._buffer_length == self._buffer_size
    
    def add(self, experience: Experience, AoI):
        priority = -AoI
        if self.is_full():
            self._buffer = np.insert(self._buffer,self._buffer_size,(priority, experience)) # push to end (newest)
            self._buffer = np.delete(self._buffer,0) # pop the first (oldest)
        else:
            self._buffer[self._buffer_length] = (priority, experience)
            self._buffer_length += 1

    def sample(self):
        # generate rank-based sampling probability sequence
        if(self._buffer_length >= MEMORY_SIZE):
            rank_ps = np.arange(MEMORY_SIZE)+1
        else:
            rank_ps = np.arange(self._buffer_length)+1
        sampling_probs = rank_ps**self._alpha / np.sum(rank_ps**self._alpha, dtype=np.int64)
        # sample transitions 
        idxs_ps = self._random_state.choice(np.arange(rank_ps.size),
                                         size=self._batch_size,
                                         replace=True,
                                         p=sampling_probs)
        
        if(self._buffer_length >= MEMORY_SIZE):
            idxs = idxs_ps + (self._buffer_length-MEMORY_SIZE)
        else:
            idxs = idxs_ps
        # compute sampling weights
        experiences = self._buffer["experience"][idxs]
        batch = Experience(*zip(*experiences))
        
        if(self._buffer_length >= MEMORY_SIZE):
            weights = (MEMORY_SIZE * sampling_probs[idxs_ps])**-self._beta
        else:
            weights = (self._buffer_length * sampling_probs[idxs_ps])**-self._beta
        normalized_weights = weights / weights.max()
                        
        return (
            torch.FloatTensor(np.array(batch.states)).to(self.device),
            torch.FloatTensor(np.array(batch.actions)).to(self.device),
            torch.FloatTensor(np.array(batch.rewards)).to(self.device),
            torch.FloatTensor(np.array(batch.next_states)).to(self.device),
            torch.FloatTensor(np.array(batch.dones)).to(self.device),
            torch.FloatTensor(np.array(idxs)).to(self.device),
            torch.FloatTensor(np.array(normalized_weights)).to(self.device)
        )
    

class PrioritizedExperienceReplayBuffer_OW:

    def __init__(self, alpha=ALPHA_IS, beta=BETA):
      
        self._batch_size = BATCH_SIZE
        self._buffer_size = MEMORY_CAPACITY
        self._buffer_length = 0 # current buffer length
        self._buffer = np.empty(self._buffer_size, dtype=[("priority", np.float32), ("experience_ow", Experience_OW)])
        
        self._alpha = alpha
        self._beta = beta
        self._random_state = np.random.RandomState()
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def buffer_len(self):
        return self._buffer_length
    
    def is_full(self) -> bool:
        return self._buffer_length == self._buffer_size
    
    def add(self, experience_ow: Experience_OW, AoI):
        priority = -AoI
        if self.is_full():
            self._buffer = np.insert(self._buffer,self._buffer_size,(priority, experience_ow)) # push to end (newest)
            self._buffer = np.delete(self._buffer,0) # pop the first (oldest)
        else:
            self._buffer[self._buffer_length] = (priority, experience_ow)
            self._buffer_length += 1

    def sample_ow(self):
        # generate rank-based sampling probability sequence
        if(self._buffer_length >= MEMORY_SIZE):
            rank_ps = np.arange(MEMORY_SIZE)+1
        else:
            rank_ps = np.arange(self._buffer_length)+1
        sampling_probs = rank_ps**self._alpha / np.sum(rank_ps**self._alpha, dtype=np.int64)
        # sample transitions
        idxs_ps = self._random_state.choice(np.arange(rank_ps.size),
                                         size=self._batch_size,
                                         replace=True,
                                         p=sampling_probs)
        
        if(self._buffer_length >= MEMORY_SIZE):
            idxs = idxs_ps + (self._buffer_length-MEMORY_SIZE)
        else:
            idxs = idxs_ps
        # compute sampling weights
        experiences_ow = self._buffer["experience_ow"][idxs]
        batch = Experience_OW(*zip(*experiences_ow))
        
        if(self._buffer_length >= MEMORY_SIZE):
            weights = (MEMORY_SIZE * sampling_probs[idxs_ps])**-self._beta
        else:
            weights = (self._buffer_length * sampling_probs[idxs_ps])**-self._beta
        normalized_weights = weights / weights.max()
                        
        return (
            torch.FloatTensor(np.array(batch.ows)).to(self.device),
            torch.FloatTensor(np.array(batch.states)).to(self.device),
            torch.FloatTensor(np.array(batch.actions)).to(self.device),
            torch.FloatTensor(np.array(batch.rewards)).to(self.device),
            torch.FloatTensor(np.array(batch.next_ows)).to(self.device),
            torch.FloatTensor(np.array(batch.next_states)).to(self.device),
            torch.FloatTensor(np.array(batch.dones)).to(self.device),
            torch.FloatTensor(np.array(idxs)).to(self.device),
            torch.FloatTensor(np.array(normalized_weights)).to(self.device)
        )
    
    # Update the ranking value of transitions 
    def update_priorities(self, idxs: np.array, priorities: np.array):
        for i in idxs:
            self._buffer["priority"][i] = float(math.floor(self._buffer["priority"][i]))
        
        priorities_sigmoid = (1/(1 + np.exp(-priorities))).round(decimals=4) - 1e-4 # bias for rounding to avoid sigmoid≈1
        self._buffer["priority"][idxs] += priorities_sigmoid
    
    # Sort transitions in the memory based on ranking value
    def sort_priorities(self):
        if(self._buffer_length >= MEMORY_SIZE):
            buffer_list = list(self._buffer[(self._buffer_length - MEMORY_SIZE):self._buffer_length])
            buffer_list.sort(key=lambda x:x[0])

            self._buffer_length = MEMORY_SIZE
        else:
            buffer_list = list(self._buffer[:self._buffer_length])
            buffer_list.sort(key=lambda x:x[0])
        
        self._buffer[:self._buffer_length] = np.array(buffer_list, 
                                                      dtype=[("priority", np.float32), ("experience_ow", Experience_OW)])

In [ ]:
'''Initialize History Buffer'''
# Controller_ObservationAction_Window: state-action history of the intelligent controller
# Scheduler_ObservationAction_Window: state-action history of the transmission scheduler


class Controller_ObservationAction_Window:

    def __init__(self):
        self._buffer_size = LEN
        self._buffer = deque()
        # padding zeros
        for i in range(LEN):
            self._buffer.append(np.zeros(ACTOR_INPUT_DIM))
            self._buffer.append(np.zeros(CONTROL_ACTION_DIM))
    
    def add(self, obervation_OR_action):
        self._buffer.append(obervation_OR_action)
        self._buffer.popleft()
        
    def read(self):
        his_obs_act = self._buffer.copy()
        his_obs_act = np.array(his_obs_act)
        his_obs_act = np.concatenate(his_obs_act, axis=None)
        return his_obs_act
    

class Scheduler_ObservationAction_Window:

    def __init__(self):
        self._buffer_size = LEN
        self._buffer = deque()
        # padding zeros
        for i in range(LEN):
            self._buffer.append(np.zeros(DQN_INPUT_DIM))
            self._buffer.append(np.zeros(SCHEDULE_ACTION_DIM))
    
    def add(self, obervation_OR_action):
        self._buffer.append(obervation_OR_action)
        self._buffer.popleft()
        
    def read(self):
        his_obs_act = self._buffer.copy()
        his_obs_act = np.array(his_obs_act)
        his_obs_act = np.concatenate(his_obs_act, axis=None)
        return his_obs_act

In [ ]:
'''Functions for Implementation'''
# channel_state_transition: Markov fading channel state
# add_gaussian_noise: add sensor measurement noise
# concatenate_controller_state: generate input of intelligent controller
# concatenate_scheduler_state: generate input of transmission scheduler

def channel_state_transition(ch):
    if(ch == 0.05):
        next_ch = np.random.choice([0.05, 0.1], p=[0.3, 0.7])
    else:
        next_ch = np.random.choice([0.05, 0.1], p=[0.7, 0.3])
    return next_ch

def add_gaussian_noise(state):
    noise = np.random.randn(state.shape[0]) 
    sensor = state + SENSOR_NOISE_STD * noise
    return sensor

def concatenate_controller_state(plant_state, ch_sc, ch_ca, AoI_sc):
    ch_state = np.array([ch_sc, ch_ca, AoI_sc], dtype=np.float32)
    state = np.concatenate((plant_state,ch_state))
    return state

def concatenate_scheduler_state(plant_state, ch_sc, AoI_sc):
    ch_state = np.array([ch_sc, AoI_sc], dtype=np.float32)
    state = np.concatenate((plant_state,ch_state))
    return state

In [ ]:
'''Evaluation of trained NNs on MuJoCo tasks'''
# Estimator
# Intelligent controller (actor, critic) 
# Transmission scheduler (DQN) 


def eval_policy(policy, env_name, seed, training_timestep, eval_episodes=EVAL_EPISODES):
    eval_env = gym.make(env_name)
    eval_env.seed(seed + 100)
    
    eval_return_buffer = []
    eval_error_buffer = []
    
    eval_scheduler_OW = Scheduler_ObservationAction_Window()
    eval_controller_OW = Controller_ObservationAction_Window()
    
    for i in range(eval_episodes):
        eval_episode_reward = 0.
        eval_episode_comm_cost = 0.
        eval_episode_error = 0.
        
        eval_timesteps = 0
        
        eval_state, eval_done = eval_env.reset(), False
        EVAL_INITIAL_OBS = add_gaussian_noise(eval_state)
        
        eval_controller_input = EVAL_INITIAL_OBS
        
        eval_scheduler_OW.__init__()
        eval_controller_OW.__init__()
        
        eval_LS = 0.05 # initial uplink (sensor-controller) channel loss rate
        eval_LA = 0.05 # initial downlink (controller-actuator) channel loss rate
        
        eval_AoI = 0
        
        eval_scheduler_concatenated_input = concatenate_scheduler_state(EVAL_INITIAL_OBS, eval_LS, 1)
        eval_schedule = np.array([1])
        eval_sc_transmission = 1
        
        # check unusual env state
        nan_signal = 0
        
        while not eval_done:
            # check unusual env state
            if(nan_signal):
                break
        
            eval_controller_concatenated_input = concatenate_controller_state(eval_controller_input, eval_LS, eval_LA, eval_AoI)
            eval_controller_concatenated_ow = eval_controller_OW.read()
              
            if(eval_sc_transmission == 0):
                eval_comm_cost_sc = 0
            else:
                eval_comm_cost_sc = COMM_COST
             
            if(np.random.rand() < eval_LA):
                eval_action = np.zeros(CONTROL_ACTION_DIM)
            else:
                eval_action = policy.select_action(np.array(eval_controller_concatenated_ow), 
                                              np.array(eval_controller_concatenated_input))
            
            eval_scheduler_OW.add(eval_scheduler_concatenated_input)
            eval_scheduler_OW.add(eval_schedule)
            
            eval_controller_OW.add(eval_controller_concatenated_input)
            eval_controller_OW.add(eval_action)
            
            # check unusual env state
            if(True in np.isnan(eval_action)):
                nan_signal = 1
                break
            
            # take 1 step
            eval_state_est = policy.predict_state(np.array(eval_controller_input), np.array(eval_action))
            
            eval_state, eval_oc_cost, eval_done, _ = eval_env.step(eval_action)
            eval_state_obs = add_gaussian_noise(eval_state)
            
            eval_reward = eval_oc_cost - eval_comm_cost_sc
        
            eval_LS = channel_state_transition(eval_LS)
            eval_LA = channel_state_transition(eval_LA) 
           
            eval_scheduler_concatenated_input = concatenate_scheduler_state(eval_state_est, eval_LS, eval_AoI+1)
            eval_scheduler_concatenated_ow = eval_scheduler_OW.read()
            
            if(training_timestep > PRE_TRAINING_TIMESTEPS):
                eval_schedule = policy.schedule_transmission(np.array(eval_scheduler_concatenated_ow), 
                                                            np.array(eval_scheduler_concatenated_input))
            else:
                eval_schedule = np.array([1])
            
            eval_sc_transmission = policy.control_transmission(eval_schedule)
            
            # next step
            if(np.random.rand() < eval_LS) or (eval_sc_transmission == 0):
                eval_controller_input = eval_state_est
                eval_AoI += 1
            else:
                eval_controller_input = eval_state_obs
                eval_AoI = 0
            
            # Evaluate reward
            eval_episode_reward += eval_reward
            
            # Evaluate prediction error
            eval_timestep_error = np.square(eval_state_obs - eval_state_est).mean()
            eval_episode_error += eval_timestep_error
            eval_timesteps += 1
        
        if not(nan_signal):
            eval_return_buffer.append(eval_episode_reward)
            eval_error_buffer.append(eval_episode_error / eval_timesteps)
    
    eval_env.close()
    
    # Average over N eval episodes
    eval_return = statistics.mean(eval_return_buffer)
    eval_error = statistics.mean(eval_error_buffer)
   
    print("---------------------------------------")
    print(f"Evaluation over {eval_episodes} episodes: {eval_return:.3f}")
    print(f"Estimation over {eval_episodes} episodes: {eval_error:.3f}")
    print("---------------------------------------")

In [ ]:
'''Main: joint training of NNs in WNCS over fading channels'''
# Estimator
# Intelligent controller (actor, critic) 
# Transmission scheduler (DQN) 


# Initialize policy
policy = Tailored_TD3()

# Initialize replay buffer
estimator_replay_buffer = PrioritizedExperienceReplayBuffer()
scheduler_replay_buffer = PrioritizedExperienceReplayBuffer_OW(ALPHA_UN, BETA)
controller_replay_buffer = PrioritizedExperienceReplayBuffer_OW()

# Evaluate untrained policy
eval_policy(policy, ENV_NAME, SEED, 0)

state, done = env.reset(), False
episode_reward = 0
episode_timesteps = 0
episode_num = 0

INITIAL_OBS = add_gaussian_noise(state)

controller_obs = None
controller_est = None

controller_next_obs = INITIAL_OBS
controller_next_est = None

LS = LS # initial uplink (sensor-controller) channel loss rate
LA = LA # initial downlink (controller-actuator) channel loss rate

signal_sc = 1 # uplink dropout signal (0: drop, 1: receive)
AoI = 0

# Initialize key variables
SCHEDULER_OW = Scheduler_ObservationAction_Window()
CONTROLLER_OW = Controller_ObservationAction_Window()

scheduler_concatenated_est = concatenate_scheduler_state(INITIAL_OBS, LS, 1)
scheduler_concatenated_ow = SCHEDULER_OW.read()
schedule = np.array([1]) 
sc_transmission = 1

# Initialize temp variables
REC_estimator_current = None
REC_estimator_next = None

REC_scheduler_current = None
REC_scheduler_next = None

REC_scheduler_current_ow = None
REC_scheduler_next_ow = None

REC_controller_current = None
REC_controller_next = None

REC_controller_current_ow = None
REC_controller_next_ow = None

for t in range(int(MAX_TIMESTEPS)):

    episode_timesteps += 1
    
    '''Packet dropout signal'''
    if (signal_sc == 0):
        '''Uplink channel'''
        controller_est = controller_next_est
        
        controller_concatenated_est = concatenate_controller_state(controller_est, LS, LA, AoI)
        controller_concatenated_ow = CONTROLLER_OW.read()
        
        REC_estimator_current = controller_est
        
        REC_scheduler_current = scheduler_concatenated_est
        REC_scheduler_current_ow = scheduler_concatenated_ow
        
        REC_controller_current = controller_concatenated_est
        REC_controller_current_ow = controller_concatenated_ow
        
        # communication cost    
        if(sc_transmission == 0):
            comm_cost_sc = 0
        else:
            comm_cost_sc = COMM_COST
        
        '''Downlink channel'''
        if(np.random.rand() < LA):
            action = np.zeros(CONTROL_ACTION_DIM)
        else:
            # Select action randomly or according to policy
            if t < TIMESTEPS_BEFORE_TRAIN:
                action = env.action_space.sample()
            else:
                action = (policy.select_action(np.array(controller_concatenated_ow), 
                                               np.array(controller_concatenated_est)) 
                          + np.random.normal(0, MAX_ACTION * EXPL_NOISE_STD, 
                                           size=CONTROL_ACTION_DIM)).clip(-MAX_ACTION, MAX_ACTION)
        
        # Update history of transmission scheduler and intelligent controller  
        SCHEDULER_OW.add(scheduler_concatenated_est)
        SCHEDULER_OW.add(schedule)
        
        CONTROLLER_OW.add(controller_concatenated_est)
        CONTROLLER_OW.add(action)
        
        # Take 1 step
        controller_next_est = policy.predict_state(np.array(controller_est), np.array(action))
        
        next_state, oc_cost, done, _ = env.step(action) 
        done_bool = float(done) if episode_timesteps < env._max_episode_steps else 0
        
        reward = oc_cost - comm_cost_sc
        
        q_value = policy.select_q(controller_concatenated_ow, controller_concatenated_est, action)
        q_reward = float(q_value)
        
        '''Markov fading channel state'''
        LS = channel_state_transition(LS)
        LA = channel_state_transition(LA)
        
        '''Transmission scheduling'''
        scheduler_concatenated_est = concatenate_scheduler_state(controller_next_est, LS, AoI+1)
        scheduler_concatenated_ow = SCHEDULER_OW.read()
        
        if(t < PRE_TRAINING_TIMESTEPS):    
            schedule = np.array([1])
        else:
            # Select schedule action with e-greedy
            if(np.random.rand() < EPSILON):
                schedule = policy.schedule_transmission(np.array(scheduler_concatenated_ow), 
                                                        np.array(scheduler_concatenated_est))
            else:
                schedule = np.array([np.random.randint(0, 2)])
        
        sc_transmission = policy.control_transmission(schedule)
        
        '''Next time step'''
        REC_scheduler_next = scheduler_concatenated_est
        REC_scheduler_next_ow = scheduler_concatenated_ow
        
        if(np.random.rand() < LS) or (sc_transmission == 0):
            # Predict state
            controller_next_est = controller_next_est
            
            AoI += 1
            
            REC_estimator_next = controller_next_est
            REC_controller_next = concatenate_controller_state(controller_next_est, LS, LA, AoI)
            REC_controller_next_ow = CONTROLLER_OW.read()
            
            # Store transition in replay buffers
            scheduler_experience = Experience_OW(REC_scheduler_current_ow, REC_scheduler_current, schedule, [q_reward], 
                                       REC_scheduler_next_ow, REC_scheduler_next, [1. - done_bool])
            controller_experience = Experience_OW(REC_controller_current_ow, REC_controller_current, action, [reward], 
                                       REC_controller_next_ow, REC_controller_next, [1. - done_bool])
            
            aoi_label = 2*AoI - 1
            scheduler_replay_buffer.add(scheduler_experience, aoi_label) # transition AOI: n/n+1
            controller_replay_buffer.add(controller_experience, aoi_label)
        else:  
            # Receive noisy observation
            controller_next_obs = add_gaussian_noise(next_state)
            
            signal_sc = 1
            
            REC_estimator_next = controller_next_obs
            REC_controller_next = concatenate_controller_state(controller_next_obs, LS, LA, 0)
            REC_controller_next_ow = CONTROLLER_OW.read()
            
            # Store transition in replay buffers
            scheduler_experience = Experience_OW(REC_scheduler_current_ow, REC_scheduler_current, schedule, [q_reward], 
                                       REC_scheduler_next_ow, REC_scheduler_next, [1. - done_bool])
            controller_experience = Experience_OW(REC_controller_current_ow, REC_controller_current, action, [reward], 
                                          REC_controller_next_ow, REC_controller_next, [1. - done_bool])
            
            aoi_label = AoI
            scheduler_replay_buffer.add(scheduler_experience, aoi_label) # transition AOI: n/0
            controller_replay_buffer.add(controller_experience, aoi_label)
    
    else: 
        # Reset AoI
        AoI = 0
        
        '''Uplink channel'''
        controller_obs = controller_next_obs
        
        controller_concatenated_obs = concatenate_controller_state(controller_obs, LS, LA, 0)
        controller_concatenated_ow = CONTROLLER_OW.read()
        
        REC_estimator_current = controller_obs
        
        REC_scheduler_current = scheduler_concatenated_est
        REC_scheduler_current_ow = scheduler_concatenated_ow
        
        REC_controller_current = controller_concatenated_obs
        REC_controller_current_ow = controller_concatenated_ow
        
        # communication cost    
        if(sc_transmission == 0):
            comm_cost_sc = 0
        else:
            comm_cost_sc = COMM_COST
        
        '''Downlink channel'''
        if(np.random.rand() < LA):
            action = np.zeros(CONTROL_ACTION_DIM)
        else:
            # Select action randomly or according to policy
            if t < TIMESTEPS_BEFORE_TRAIN:
                action = env.action_space.sample()
            else:
                action = (policy.select_action(np.array(controller_concatenated_ow), 
                                               np.array(controller_concatenated_obs)) 
                          + np.random.normal(0, MAX_ACTION * EXPL_NOISE_STD, 
                                             size=CONTROL_ACTION_DIM)).clip(-MAX_ACTION, MAX_ACTION)
        
        # Update history of transmission scheduler and intelligent controller 
        SCHEDULER_OW.add(scheduler_concatenated_est)
        SCHEDULER_OW.add(schedule)
        
        CONTROLLER_OW.add(controller_concatenated_obs)
        CONTROLLER_OW.add(action)
        
        # Take 1 step
        controller_next_est = policy.predict_state(np.array(controller_obs), np.array(action))
        
        next_state, oc_cost, done, _ = env.step(action) 
        done_bool = float(done) if episode_timesteps < env._max_episode_steps else 0
        
        reward = oc_cost - comm_cost_sc
        
        q_value = policy.select_q(controller_concatenated_ow, controller_concatenated_obs, action)
        q_reward = float(q_value)
        
        '''Markov fading channel state'''
        LS = channel_state_transition(LS)
        LA = channel_state_transition(LA) 
        
        '''Transmission scheduling'''
        scheduler_concatenated_est = concatenate_scheduler_state(controller_next_est, LS, AoI+1)
        scheduler_concatenated_ow = SCHEDULER_OW.read()
        
        if(t < PRE_TRAINING_TIMESTEPS):    
            schedule = np.array([1])
        else:
            # Select schedule action with e-greedy
            if(np.random.rand() < EPSILON):
                schedule = policy.schedule_transmission(np.array(scheduler_concatenated_ow), 
                                                        np.array(scheduler_concatenated_est))
            else:
                schedule = np.array([np.random.randint(0, 2)])
        
        sc_transmission = policy.control_transmission(schedule)
        
        '''Next time step'''
        REC_scheduler_next = scheduler_concatenated_est
        REC_scheduler_next_ow = scheduler_concatenated_ow
        
        if(np.random.rand() < LS) or (sc_transmission == 0):
            # Predict state
            controller_next_est = controller_next_est
            
            signal_sc = 0
            AoI += 1
            
            REC_estimator_next = controller_next_est
            REC_controller_next = concatenate_controller_state(controller_next_est, LS, LA, 1)
            REC_controller_next_ow = CONTROLLER_OW.read()
            
            # Store transition in replay buffer
            scheduler_experience = Experience_OW(REC_scheduler_current_ow, REC_scheduler_current, schedule, [q_reward], 
                                       REC_scheduler_next_ow, REC_scheduler_next, [1. - done_bool])
            controller_experience = Experience_OW(REC_controller_current_ow, REC_controller_current, action, [reward], 
                                          REC_controller_next_ow, REC_controller_next, [1. - done_bool])
            
            aoi_label = 1
            scheduler_replay_buffer.add(scheduler_experience, aoi_label) # transition AOI: 0/1
            controller_replay_buffer.add(controller_experience, aoi_label)
        else:
            # Receive noisy observation
            controller_next_obs = add_gaussian_noise(next_state)
            
            REC_estimator_next = controller_next_obs
            REC_controller_next = concatenate_controller_state(controller_next_obs, LS, LA, 0)
            REC_controller_next_ow = CONTROLLER_OW.read()
            
            # Store transition in replay buffers
            estimator_experience = Experience(REC_estimator_current, action, [reward], 
                                              REC_estimator_next, [1. - done_bool])
            
            scheduler_experience = Experience_OW(REC_scheduler_current_ow, REC_scheduler_current, schedule, [q_reward], 
                                       REC_scheduler_next_ow, REC_scheduler_next, [1. - done_bool])
            controller_experience = Experience_OW(REC_controller_current_ow, REC_controller_current, action, [reward], 
                                          REC_controller_next_ow, REC_controller_next, [1. - done_bool])
            
            aoi_label = 0
            estimator_replay_buffer.add(estimator_experience, aoi_label) # transition AOI: 0/0 
            scheduler_replay_buffer.add(scheduler_experience, aoi_label) 
            controller_replay_buffer.add(controller_experience, aoi_label)

    state = next_state
    episode_reward += reward

    # Train agent after collecting sufficient data
    if t >= TIMESTEPS_BEFORE_TRAIN:
        policy.train(estimator_replay_buffer, scheduler_replay_buffer, controller_replay_buffer)

    if done: 
        print(f"Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
        
        '''Reset variables'''
        state, done = env.reset(), False
        episode_reward = 0
        episode_timesteps = 0
        episode_num += 1 
        
        INITIAL_OBS = add_gaussian_noise(state)
        
        controller_obs = None
        controller_est = None
        
        controller_next_obs = INITIAL_OBS
        controller_next_est = None
        
        LS = 0.05
        LA = 0.05
        
        signal_sc = 1
        AoI = 0
        
        SCHEDULER_OW.__init__()
        CONTROLLER_OW.__init__()
        
        scheduler_concatenated_est = concatenate_scheduler_state(INITIAL_OBS, LS, 1)
        scheduler_concatenated_ow = SCHEDULER_OW.read()
        schedule = np.array([1]) 
        sc_transmission = 1
        
        REC_estimator_current = None
        REC_estimator_next = None
        
        REC_scheduler_current = None
        REC_scheduler_next = None

        REC_scheduler_current_ow = None
        REC_scheduler_next_ow = None

        REC_controller_current = None
        REC_controller_next = None

        REC_controller_current_ow = None
        REC_controller_next_ow = None

    '''Evaluate trained NNs'''
    if (t + 1) % EVAL_FREQ == 0:
        eval_policy(policy, ENV_NAME, SEED, t + 1)